**Pytorch BERT baseline**

In this version, I convert https://www.kaggle.com/akensert/bert-base-tf2-0-minimalistic into pytorch version

**Please upvote the kernel if you find it helpful**

### Install HuggingFace transformers & sacremoses dependency

As we are not allowed to use internet I've created required datasets and commands to setup Hugging Face Transformers setup in offline mode. You can find the required github codebases in the datasets.

* sacremoses dependency - https://www.kaggle.com/axel81/sacremoses
* transformers - https://www.kaggle.com/axel81/transformers

In [1]:
!pip install ../input/sacremoses/sacremoses-master/
!pip install ../input/transformers/transformers-master/

Processing /kaggle/input/sacremoses/sacremoses-master
  Created wheel for sacremoses: filename=sacremoses-0.0.35-cp36-none-any.whl size=882724 sha256=85433273cfb6fb235ef52b042ba00aac744fd1d438e9a5e634874f72caacd3e2
  Stored in directory: /tmp/.cache/pip/wheels/82/48/4b/05cb49d913a40c9d76f97931cd747d72fb17a77b0f6415cdba
Successfully built sacremoses
Processing /kaggle/input/transformers/transformers-master
  Created wheel for transformers: filename=transformers-2.1.1-cp36-none-any.whl size=334890 sha256=b0c679acb118e9ee93839138bdfc8d67d7d80282e6ba1ca140aa75dee5d0aa1d
  Stored in directory: /tmp/.cache/pip/wheels/ce/f3/1a/ee7248890cb4b8e8975988b1a67999e2d09ef54ce8ee815255
Successfully built transformers


### Required Imports

I've added imports that will be used in training too

In [2]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
DATA_DIR = '../input/google-quest-challenge'

In [3]:
!ls ../input

google-quest-challenge		    qabertbasebce10fold  transformers
pretrained-bert-models-for-pytorch  sacremoses


In [4]:
os.listdir("../input/qabertbasebce10fold")

['fold_9_checkpoint.pth',
 'fold_1_checkpoint.pth',
 'fold_4_checkpoint.pth',
 'fold_0_checkpoint.pth',
 'fold_2_checkpoint.pth',
 'fold_8_checkpoint.pth',
 'fold_3_checkpoint.pth',
 'fold_6_checkpoint.pth',
 'fold_5_checkpoint.pth',
 'fold_7_checkpoint.pth']

In [5]:
sub = pd.read_csv(f'{DATA_DIR}/sample_submission.csv')
sub.head()

,qa_id,question_asker_intent_understanding,question_body_critical,question_conversational,question_expect_short_answer,question_fact_seeking,question_has_commonly_accepted_answer,question_interestingness_others,question_interestingness_self,question_multi_intent,...,question_well_written,answer_helpful,answer_level_of_information,answer_plausible,answer_relevance,answer_satisfaction,answer_type_instructions,answer_type_procedure,answer_type_reason_explanation,answer_well_written
0,39,0.00308,0.00308,0.00308,0.00308,0.00308,0.00308,0.00308,0.00308,0.00308,...,0.00308,0.00308,0.00308,0.00308,0.00308,0.00308,0.00308,0.00308,0.00308,0.00308
1,46,0.00448,0.00448,0.00448,0.00448,0.00448,0.00448,0.00448,0.00448,0.00448,...,0.00448,0.00448,0.00448,0.00448,0.00448,0.00448,0.00448,0.00448,0.00448,0.00448
2,70,0.00673,0.00673,0.00673,0.00673,0.00673,0.00673,0.00673,0.00673,0.00673,...,0.00673,0.00673,0.00673,0.00673,0.00673,0.00673,0.00673,0.00673,0.00673,0.00673
3,132,0.01401,0.01401,0.01401,0.01401,0.01401,0.01401,0.01401,0.01401,0.01401,...,0.01401,0.01401,0.01401,0.01401,0.01401,0.01401,0.01401,0.01401,0.01401,0.01401
4,200,0.02074,0.02074,0.02074,0.02074,0.02074,0.02074,0.02074,0.02074,0.02074,...,0.02074,0.02074,0.02074,0.02074,0.02074,0.02074,0.02074,0.02074,0.02074,0.02074


In [6]:
TARGET_COLUMNS = sub.columns.values[1:].tolist()
TARGET_COLUMNS

['question_asker_intent_understanding',
 'question_body_critical',
 'question_conversational',
 'question_expect_short_answer',
 'question_fact_seeking',
 'question_has_commonly_accepted_answer',
 'question_interestingness_others',
 'question_interestingness_self',
 'question_multi_intent',
 'question_not_really_a_question',
 'question_opinion_seeking',
 'question_type_choice',
 'question_type_compare',
 'question_type_consequence',
 'question_type_definition',
 'question_type_entity',
 'question_type_instructions',
 'question_type_procedure',
 'question_type_reason_explanation',
 'question_type_spelling',
 'question_well_written',
 'answer_helpful',
 'answer_level_of_information',
 'answer_plausible',
 'answer_relevance',
 'answer_satisfaction',
 'answer_type_instructions',
 'answer_type_procedure',
 'answer_type_reason_explanation',
 'answer_well_written']

### Define dataset

In [7]:
train = pd.read_csv(f'{DATA_DIR}/train.csv')
train.head()

,qa_id,question_title,question_body,question_user_name,question_user_page,answer,answer_user_name,answer_user_page,url,category,...,question_well_written,answer_helpful,answer_level_of_information,answer_plausible,answer_relevance,answer_satisfaction,answer_type_instructions,answer_type_procedure,answer_type_reason_explanation,answer_well_written
0,0,What am I losing when using extension tubes in...,After playing around with macro photography on...,ysap,https://photo.stackexchange.com/users/1024,"I just got extension tubes, so here's the skin...",rfusca,https://photo.stackexchange.com/users/1917,http://photo.stackexchange.com/questions/9169/...,LIFE_ARTS,...,1.000000,1.000000,0.666667,1.000000,1.000000,0.800000,1.0,0.000000,0.000000,1.000000
1,1,What is the distinction between a city and a s...,I am trying to understand what kinds of places...,russellpierce,https://rpg.stackexchange.com/users/8774,It might be helpful to look into the definitio...,Erik Schmidt,https://rpg.stackexchange.com/users/1871,http://rpg.stackexchange.com/questions/47820/w...,CULTURE,...,0.888889,0.888889,0.555556,0.888889,0.888889,0.666667,0.0,0.000000,0.666667,0.888889
2,2,Maximum protusion length for through-hole comp...,I'm working on a PCB that has through-hole com...,Joe Baker,https://electronics.stackexchange.com/users/10157,Do you even need grooves? We make several pro...,Dwayne Reid,https://electronics.stackexchange.com/users/64754,http://electronics.stackexchange.com/questions...,SCIENCE,...,0.777778,0.777778,0.555556,1.000000,1.000000,0.666667,0.0,0.333333,1.000000,0.888889
3,3,Can an affidavit be used in Beit Din?,"An affidavit, from what i understand, is basic...",Scimonster,https://judaism.stackexchange.com/users/5151,"Sending an ""affidavit"" it is a dispute between...",Y e z,https://judaism.stackexchange.com/users/4794,http://judaism.stackexchange.com/questions/551...,CULTURE,...,0.888889,0.833333,0.333333,0.833333,1.000000,0.800000,0.0,0.000000,1.000000,1.000000
4,5,How do you make a binary image in Photoshop?,I am trying to make a binary image. I want mor...,leigero,https://graphicdesign.stackexchange.com/users/...,Check out Image Trace in Adobe Illustrator. \n...,q2ra,https://graphicdesign.stackexchange.com/users/...,http://graphicdesign.stackexchange.com/questio...,LIFE_ARTS,...,1.000000,1.000000,0.666667,1.000000,1.000000,0.800000,1.0,0.000000,1.000000,1.000000


In [8]:
test = pd.read_csv(f'{DATA_DIR}/test.csv')
test.head()

,qa_id,question_title,question_body,question_user_name,question_user_page,answer,answer_user_name,answer_user_page,url,category,host
0,39,Will leaving corpses lying around upset my pri...,I see questions/information online about how t...,Dylan,https://gaming.stackexchange.com/users/64471,There is no consequence for leaving corpses an...,Nelson868,https://gaming.stackexchange.com/users/97324,http://gaming.stackexchange.com/questions/1979...,CULTURE,gaming.stackexchange.com
1,46,Url link to feature image in the portfolio,I am new to Wordpress. i have issue with Featu...,Anu,https://wordpress.stackexchange.com/users/72927,I think it is possible with custom fields.\n\n...,Irina,https://wordpress.stackexchange.com/users/27233,http://wordpress.stackexchange.com/questions/1...,TECHNOLOGY,wordpress.stackexchange.com
2,70,"Is accuracy, recoil or bullet spread affected ...","To experiment I started a bot game, toggled in...",Konsta,https://gaming.stackexchange.com/users/37545,You do not have armour in the screenshots. Thi...,Damon Smithies,https://gaming.stackexchange.com/users/70641,http://gaming.stackexchange.com/questions/2154...,CULTURE,gaming.stackexchange.com
3,132,Suddenly got an I/O error from my external HDD,I have used my Raspberry Pi as a torrent-serve...,robbannn,https://raspberrypi.stackexchange.com/users/17341,Your Western Digital hard drive is disappearin...,HeatfanJohn,https://raspberrypi.stackexchange.com/users/1311,http://raspberrypi.stackexchange.com/questions...,TECHNOLOGY,raspberrypi.stackexchange.com
4,200,Passenger Name - Flight Booking Passenger only...,I have bought Delhi-London return flights for ...,Amit,https://travel.stackexchange.com/users/29089,I called two persons who work for Saudia (tick...,Nean Der Thal,https://travel.stackexchange.com/users/10051,http://travel.stackexchange.com/questions/4704...,CULTURE,travel.stackexchange.com


In [9]:
import torch
#import torch.utils.data as data
from torchvision import datasets, models, transforms
from transformers import *
from sklearn.utils import shuffle
import random
from math import floor, ceil
from sklearn.model_selection import GroupKFold

MAX_LEN = 512
#MAX_Q_LEN = 250
#MAX_A_LEN = 259
SEP_TOKEN_ID = 102

class QuestDataset(torch.utils.data.Dataset):
    def __init__(self, df, model_type="bert-base-uncased", train_mode=True, labeled=True):
        self.df = df
        self.train_mode = train_mode
        self.labeled = labeled
        self.tokenizer = BertTokenizer.from_pretrained('../input/pretrained-bert-models-for-pytorch/bert-base-uncased-vocab.txt',do_lower_case='uncased')

    def __getitem__(self, index):
        row = self.df.iloc[index]
        token_ids, seg_ids = self.get_token_ids(row)
        if self.labeled:
            labels = self.get_label(row)
            return token_ids, seg_ids, labels
        else:
            return token_ids, seg_ids

    def __len__(self):
        return len(self.df)

    def select_tokens(self, tokens, max_num):
        if len(tokens) <= max_num:
            return tokens
        if self.train_mode:
            num_remove = len(tokens) - max_num
            remove_start = random.randint(0, len(tokens)-num_remove-1)
            return tokens[:remove_start] + tokens[remove_start + num_remove:]
        else:
            return tokens[:max_num//2] + tokens[-(max_num - max_num//2):]

    def trim_input(self, title, question, answer, max_sequence_length=MAX_LEN, 
                t_max_len=30, q_max_len=239, a_max_len=239):
        t = self.tokenizer.tokenize(title)
        q = self.tokenizer.tokenize(question)
        a = self.tokenizer.tokenize(answer)

        t_len = len(t)
        q_len = len(q)
        a_len = len(a)

        if (t_len+q_len+a_len+4) > max_sequence_length:

            if t_max_len > t_len:
                t_new_len = t_len
                a_max_len = a_max_len + floor((t_max_len - t_len)/2)
                q_max_len = q_max_len + ceil((t_max_len - t_len)/2)
            else:
                t_new_len = t_max_len

            if a_max_len > a_len:
                a_new_len = a_len 
                q_new_len = q_max_len + (a_max_len - a_len)
            elif q_max_len > q_len:
                a_new_len = a_max_len + (q_max_len - q_len)
                q_new_len = q_len
            else:
                a_new_len = a_max_len
                q_new_len = q_max_len


            if t_new_len+a_new_len+q_new_len+4 != max_sequence_length:
                raise ValueError("New sequence length should be %d, but is %d" 
                                 % (max_sequence_length, (t_new_len+a_new_len+q_new_len+4)))

            t = t[:t_new_len]
            q = q[:q_new_len]
            a = a[:a_new_len]

        return t, q, a
        
    def get_token_ids(self, row):
        t_tokens, q_tokens, a_tokens = self.trim_input(row.question_title, row.question_body, row.answer)

        tokens = ['[CLS]'] + t_tokens + ['[SEP]'] + q_tokens + ['[SEP]'] + a_tokens + ['[SEP]']
        token_ids = self.tokenizer.convert_tokens_to_ids(tokens)
        if len(token_ids) < MAX_LEN:
            token_ids += [0] * (MAX_LEN - len(token_ids))
        ids = torch.tensor(token_ids)
        seg_ids = self.get_seg_ids(ids)
        return ids, seg_ids
    
    def get_seg_ids(self, ids):
        seg_ids = torch.zeros_like(ids)
        seg_idx = 0
        first_sep = True
        for i, e in enumerate(ids):
            seg_ids[i] = seg_idx
            if e == SEP_TOKEN_ID:
                if first_sep:
                    first_sep = False
                else:
                    seg_idx = 1
        pad_idx = torch.nonzero(ids == 0)
        seg_ids[pad_idx] = 0

        return seg_ids

    def get_label(self, row):
        #print(row[TARGET_COLUMNS].values)
        return torch.tensor(row[TARGET_COLUMNS].values.astype(np.float32))

    def collate_fn(self, batch):
        token_ids = torch.stack([x[0] for x in batch])
        seg_ids = torch.stack([x[1] for x in batch])
    
        if self.labeled:
            labels = torch.stack([x[2] for x in batch])
            return token_ids, seg_ids, labels
        else:
            return token_ids, seg_ids

def get_test_loader(batch_size=4):
    df = pd.read_csv(f'{DATA_DIR}/test.csv')
    ds_test = QuestDataset(df, train_mode=False, labeled=False)
    loader = torch.utils.data.DataLoader(ds_test, batch_size=batch_size, shuffle=False, num_workers=0, collate_fn=ds_test.collate_fn, drop_last=False)
    loader.num = len(df)
    
    return loader
        
def get_train_val_loaders(batch_size=4, val_batch_size=4, ifold=0):
    df = pd.read_csv(f'{DATA_DIR}/train.csv')
    df = shuffle(df, random_state=42)
    #split_index = int(len(df) * (1-val_percent))
    gkf = GroupKFold(n_splits=5).split(X=df.question_body, groups=df.question_body)
    for fold, (train_idx, valid_idx) in enumerate(gkf):
        if fold == ifold:
            df_train = df.iloc[train_idx]
            df_val = df.iloc[valid_idx]
            break

    #print(df_val.head())
    #df_train = df[:split_index]
    #df_val = df[split_index:]

    print(df_train.shape)
    print(df_val.shape)

    ds_train = QuestDataset(df_train)
    train_loader = torch.utils.data.DataLoader(ds_train, batch_size=batch_size, shuffle=True, num_workers=0, collate_fn=ds_train.collate_fn, drop_last=True)
    train_loader.num = len(df_train)

    ds_val = QuestDataset(df_val, train_mode=False)
    val_loader = torch.utils.data.DataLoader(ds_val, batch_size=val_batch_size, shuffle=False, num_workers=0, collate_fn=ds_val.collate_fn, drop_last=False)
    val_loader.num = len(df_val)
    val_loader.df = df_val

    return train_loader, val_loader

def test_train_loader():
    loader, _ = get_train_val_loaders(4, 4, 1)
    for ids, seg_ids, labels in loader:
        print(ids)
        print(seg_ids.numpy())
        print(labels)
        break
def test_test_loader():
    loader = get_test_loader(4)
    for ids, seg_ids in loader:
        print(ids)
        print(seg_ids)
        break

In [10]:
test_train_loader()

(4863, 41)
(1216, 41)
tensor([[ 101, 2054, 2003,  ...,    0,    0,    0],
        [ 101, 2079, 4641,  ...,    0,    0,    0],
        [ 101, 2003, 2045,  ...,    0,    0,    0],
        [ 101, 2054, 2024,  ..., 3746, 1024,  102]])
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 1 1 1]]
tensor([[1.0000, 1.0000, 0.0000, 0.3333, 0.6667, 0.6667, 0.6667, 0.5556, 0.0000,
         0.0000, 1.0000, 0.0000, 1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         1.0000, 0.0000, 1.0000, 0.8889, 0.5556, 1.0000, 1.0000, 0.8667, 0.0000,
         0.0000, 1.0000, 1.0000],
        [0.8889, 0.6667, 0.0000, 0.6667, 1.0000, 0.6667, 0.4444, 0.4444, 1.0000,
         0.0000, 0.3333, 1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.3333, 0.3333,
         0.3333, 0.0000, 0.8889, 1.0000, 0.8889, 1.0000, 1.0000, 1.0000, 0.0000,
         0.0000, 1.0000, 1.0000],
        [0.6667, 0.6667, 0.0000, 1.0000, 1.0000, 1.0000, 0.5556, 0.4444, 0.0000,
         0.0000, 0.0000, 0.3333, 0.0000, 0.0000, 0.0000, 

## Build Model

In [11]:
from transformers import *
import torch
import torch.nn as nn
import torch.nn.functional as F

bert_model_config = '../input/pretrained-bert-models-for-pytorch/bert-base-uncased/bert_config.json'
bert_config = BertConfig.from_json_file(bert_model_config)
bert_config.output_hidden_states = True

class QuestModel(nn.Module):
    def __init__(self, model_type="bert-base-uncased", n_classes=30):
        super(QuestModel, self).__init__()
        self.model_name = 'QuestModel'
        model_path = os.path.join('../input/pretrained-bert-models-for-pytorch/' + model_type)
        self.bert_model = BertModel.from_pretrained(model_path, config=bert_config)   
        if model_type == "bert-base-uncased":
            self.fc = nn.Linear(768 * 1, n_classes)
            # self.fc_1 = nn.Linear(768 * 1, 512)
            # self.fc_2 = nn.Linear(512, n_classes)
        elif model_type == "bert-large-uncased":
            self.fc = nn.Linear(1024 * 1, n_classes)
            # self.fc_1 = nn.Linear(1024 * 1, 512)
            # self.fc_2 = nn.Linear(512, n_classes)
        elif model_type == "bert-base-cased":
            self.fc = nn.Linear(768 * 1, n_classes)
            # self.fc_1 = nn.Linear(768 * 1, 512)
            # self.fc_2 = nn.Linear(512, n_classes)
        else:
            raise NotImplementedError
        self.selu = nn.SELU()
        self.relu = nn.ReLU()
        self.tanh = nn.Tanh()
        self.dropouts = nn.ModuleList([
            nn.Dropout(0.5) for _ in range(5)
        ])

    def forward(self, ids, seg_ids):
        attention_mask = (ids > 0)
        outputs = self.bert_model(input_ids=ids, token_type_ids=seg_ids, attention_mask=attention_mask)
        
        sequence_out = outputs[0] # N * 512 * 768
        # pooled_out = outputs[1] #  N * 768
        
        # use sequence_out + global_average_pooling
        avg_out = torch.mean(sequence_out, dim=1, keepdim=True)

        # use sequence_out + global_average_pooling cat sequence_out + global_max_pooling
        # out_mean = torch.squeeze(torch.mean(sequence_out, dim=1))
        # out_max, _ = torch.max(sequence_out, dim=1)
        # out = torch.cat([out_mean, out_max], dim=1)
        
        # sequence_out = outputs[0]
        # out = torch.squeeze(torch.mean(sequence_out[:, -4:, :], dim=1))
        
        hidden_states = outputs[2]
        # print(len(hidden_states)) 
        # 13 (embedding + 12 transformers) for base, 
        # 26 (embedding + 25 transformers) for large, 
        # we choose last 4 heads not including pooler
#         h1 = self.tanh(hidden_states[-1][:, 0].reshape((-1, 1, 768)))
        h2 = torch.mean(hidden_states[-2], dim=1, keepdim=True).reshape((-1, 1, 768))
        h3 = torch.mean(hidden_states[-3], dim=1, keepdim=True).reshape((-1, 1, 768))
        h4 = torch.mean(hidden_states[-4], dim=1, keepdim=True).reshape((-1, 1, 768))

#         avg_h = torch.cat([h2, h3, h4], 1)
#         out = self.tanh(torch.mean(torch.cat([avg_out, h2, h3, h4], 1), dim=1))
        out = torch.mean(torch.cat([avg_out, h2, h3, h4], 1), dim=1)
#         out = torch.squeeze(avg_out)
        
        for i, dropout in enumerate(self.dropouts):
            if i == 0:
                logit = self.fc(dropout(out))
            else:
                logit += self.fc(dropout(out))
                
        return logit / len(self.dropouts)
        
        # for i, dropout in enumerate(self.dropouts):
        #     if i == 0:
        #         logit = self.fc_1(dropout(out))
        #     else:
        #         logit += self.fc_1(dropout(out))
        
        # logit /= len(self.dropouts)
        
        # return  self.fc_2(self.relu(logit))
    
def test_model():
    x = torch.tensor([[1,2,3,4,5, 0, 0], [1,2,3,4,5, 0, 0]])
    seg_ids = torch.tensor([[0,0,0,0,0, 0, 0], [0,0,0,0,0, 0, 0]])
    model = QuestModel()

    y = model(x, seg_ids)
    print(y)

In [12]:
def create_model(model_file):
    model = QuestModel()
    model = nn.DataParallel(model)
    model.load_state_dict(torch.load(model_file))
    model = model.cuda()
    return model

def create_models():
    models = []
    for i in range(10):
        model = create_model(f'../input/qabertbasebce10fold/fold_{i}_checkpoint.pth')
        model.eval()
        models.append(model)
    return models

In [13]:
from tqdm import tqdm
import torch
def predict(models, test_loader):
    all_scores = []
    with torch.no_grad():
        for ids, seg_ids in tqdm(test_loader, total=test_loader.num // test_loader.batch_size):
            ids, seg_ids = ids.cuda(), seg_ids.cuda()
            scores = []
            for model in models:
                outputs = torch.sigmoid(model(ids, seg_ids)).cpu()
                scores.append(outputs)
            all_scores.append(torch.mean(torch.stack(scores), 0))

    all_scores = torch.cat(all_scores, 0).numpy()
    
    return all_scores

In [14]:
test_loader = get_test_loader(batch_size=32)

In [15]:
models = create_models()

In [16]:
preds = predict(models, test_loader)

15it [01:38,  6.58s/it]


In [17]:
preds_round = np.round(preds * 90) / 90 

In [18]:
preds_round[:1]

array([[0.93333334, 0.6888889 , 0.2888889 , 0.43333334, 0.6       ,
        0.5       , 0.67777777, 0.65555555, 0.6       , 0.        ,
        0.7111111 , 0.53333336, 0.02222222, 0.12222222, 0.01111111,
        0.01111111, 0.08888889, 0.13333334, 0.8       , 0.        ,
        0.93333334, 0.9111111 , 0.56666666, 0.95555556, 0.95555556,
        0.8       , 0.04444445, 0.05555556, 0.9       , 0.93333334]],
      dtype=float32)

In [19]:
preds[:1]

array([[0.93792504, 0.6847076 , 0.29084885, 0.43215036, 0.6052612 ,
        0.49569783, 0.67474043, 0.65685403, 0.6017674 , 0.0045961 ,
        0.70798963, 0.53015214, 0.01941806, 0.12060539, 0.01378705,
        0.01579896, 0.08960678, 0.1295062 , 0.79602027, 0.00325517,
        0.9364565 , 0.909567  , 0.57102066, 0.95117855, 0.9554515 ,
        0.795815  , 0.0432507 , 0.05407353, 0.90126306, 0.92850363]],
      dtype=float32)

### Generate Submission

In [20]:
sub[TARGET_COLUMNS] = preds_round

In [21]:
sub.to_csv('submission.csv', index=False)

In [22]:
sub.head()

,qa_id,question_asker_intent_understanding,question_body_critical,question_conversational,question_expect_short_answer,question_fact_seeking,question_has_commonly_accepted_answer,question_interestingness_others,question_interestingness_self,question_multi_intent,...,question_well_written,answer_helpful,answer_level_of_information,answer_plausible,answer_relevance,answer_satisfaction,answer_type_instructions,answer_type_procedure,answer_type_reason_explanation,answer_well_written
0,39,0.933333,0.688889,0.288889,0.433333,0.600000,0.500000,0.677778,0.655556,0.600000,...,0.933333,0.911111,0.566667,0.955556,0.955556,0.800000,0.044444,0.055556,0.900000,0.933333
1,46,0.911111,0.588889,0.000000,0.711111,0.844444,0.877778,0.600000,0.488889,0.077778,...,0.766667,0.955556,0.655556,0.977778,0.977778,0.888889,0.944444,0.133333,0.044444,0.900000
2,70,0.922222,0.633333,0.022222,0.755556,0.911111,0.922222,0.644444,0.533333,0.155556,...,0.866667,0.944444,0.611111,0.977778,0.977778,0.866667,0.066667,0.044444,0.900000,0.922222
3,132,0.900000,0.388889,0.000000,0.722222,0.777778,0.900000,0.522222,0.411111,0.077778,...,0.700000,0.955556,0.688889,0.977778,0.988889,0.888889,0.911111,0.144444,0.311111,0.900000
4,200,0.955556,0.488889,0.022222,0.833333,0.777778,0.822222,0.644444,0.588889,0.122222,...,0.800000,0.922222,0.611111,0.977778,0.977778,0.833333,0.344444,0.077778,0.622222,0.922222
